# Opisivanje slika sa LSTM-om
U prethodnoj vježbi ste implementirali vanila RNN i primijenili je na opisivanje slika. U ovoj vježbi ćete implementirati LSTM i iskoristiti je za opisivanje slika.

In [ ]:
import time, os, json
import numpy as np
import matplotlib.pyplot as plt

from funkcije.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from funkcije.rnn_layers import *
from funkcije.captioning_solver import CaptioningSolver
from funkcije.classifiers.rnn import CaptioningRNN
from funkcije.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from funkcije.image_utils import image_from_url

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ vraća relativnu grešku """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Učitavanje MS-COCO podataka
Kao u prethodnoj vježbi, koristićemo Microsoft COCO dataset za opisivanje.

In [ ]:
# Učitavanje COCO podataka sa diska; ovo vraća rečnik
# Mi ćemo raditi sa dimenziono smanjenim karakteristikama, ali osjećajte 
# se slobodnim da eksperimentišete sa originalnim karakteristikama mijenjajući indikator ispod. 
data = load_coco_data(pca_features=True)

# Štampajte sve ključeve i vrijednosti iz rečnika podataka
for k, v in data.items():
    if type(v) == np.ndarray:
        print(k, type(v), v.shape, v.dtype)
    else:
        print(k, type(v), len(v))

# LSTM: korak unaprijed 
Implementirajte prolaz unaprijed za jedan vremenski korak LSTM u funkciji `lstm_step_forward` u fajlu `funkcije/rnn_layers.py`. Ovo bi trebalo da bude slično `rnn_step_forward` funkciji koju ste implementirali ranije, ali koristeći LSTM.

Kada završite, pokrenite sledeću ćeliju da biste testirali vašu implementaciju. Trebalo bi da greška bude reda `e-8` ili manje.

In [ ]:
N, D, H = 3, 4, 5
x = np.linspace(-0.4, 1.2, num=N*D).reshape(N, D)
prev_h = np.linspace(-0.3, 0.7, num=N*H).reshape(N, H)
prev_c = np.linspace(-0.4, 0.9, num=N*H).reshape(N, H)
Wx = np.linspace(-2.1, 1.3, num=4*D*H).reshape(D, 4 * H)
Wh = np.linspace(-0.7, 2.2, num=4*H*H).reshape(H, 4 * H)
b = np.linspace(0.3, 0.7, num=4*H)

next_h, next_c, cache = lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)

expected_next_h = np.asarray([
    [ 0.24635157,  0.28610883,  0.32240467,  0.35525807,  0.38474904],
    [ 0.49223563,  0.55611431,  0.61507696,  0.66844003,  0.7159181 ],
    [ 0.56735664,  0.66310127,  0.74419266,  0.80889665,  0.858299  ]])
expected_next_c = np.asarray([
    [ 0.32986176,  0.39145139,  0.451556,    0.51014116,  0.56717407],
    [ 0.66382255,  0.76674007,  0.87195994,  0.97902709,  1.08751345],
    [ 0.74192008,  0.90592151,  1.07717006,  1.25120233,  1.42395676]])

print('next_h greška: ', rel_error(expected_next_h, next_h))
print('next_c greška: ', rel_error(expected_next_c, next_c))

# LSTM: korak unazad
Implementirajte prolaz unazad za jedan vremenski korak LSTM-a u funkciji `lstm_step_backward` u fajlu `funkcije/rnn_layers.py`. Jednom kada završite, pokrenite sledeću ćeliju da provjerite vašu implementaciju. Trebalo bi da greška bude reda `e-7` ili manje.

In [ ]:
np.random.seed(231)

N, D, H = 4, 5, 6
x = np.random.randn(N, D)
prev_h = np.random.randn(N, H)
prev_c = np.random.randn(N, H)
Wx = np.random.randn(D, 4 * H)
Wh = np.random.randn(H, 4 * H)
b = np.random.randn(4 * H)

next_h, next_c, cache = lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)

dnext_h = np.random.randn(*next_h.shape)
dnext_c = np.random.randn(*next_c.shape)

fx_h = lambda x: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fh_h = lambda h: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fc_h = lambda c: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fWx_h = lambda Wx: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fWh_h = lambda Wh: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fb_h = lambda b: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]

fx_c = lambda x: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fh_c = lambda h: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fc_c = lambda c: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fWx_c = lambda Wx: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fWh_c = lambda Wh: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fb_c = lambda b: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]

num_grad = eval_numerical_gradient_array

dx_num = num_grad(fx_h, x, dnext_h) + num_grad(fx_c, x, dnext_c)
dh_num = num_grad(fh_h, prev_h, dnext_h) + num_grad(fh_c, prev_h, dnext_c)
dc_num = num_grad(fc_h, prev_c, dnext_h) + num_grad(fc_c, prev_c, dnext_c)
dWx_num = num_grad(fWx_h, Wx, dnext_h) + num_grad(fWx_c, Wx, dnext_c)
dWh_num = num_grad(fWh_h, Wh, dnext_h) + num_grad(fWh_c, Wh, dnext_c)
db_num = num_grad(fb_h, b, dnext_h) + num_grad(fb_c, b, dnext_c)

dx, dh, dc, dWx, dWh, db = lstm_step_backward(dnext_h, dnext_c, cache)

print('dx greška: ', rel_error(dx_num, dx))
print('dh greška: ', rel_error(dh_num, dh))
print('dc greška: ', rel_error(dc_num, dc))
print('dWx greška: ', rel_error(dWx_num, dWx))
print('dWh greška: ', rel_error(dWh_num, dWh))
print('db greška: ', rel_error(db_num, db))

# LSTM: prolaz unaprijed
U funkciji `lstm_forward` u fajlu `funkcije/rnn_layers.py`, implementirajte `lstm_forward` funkciju da biste pokrenuli LSTM prolaz unaprijed na cijeli vremenski interval.

Kada završite, pokrenite sledeću ćeliju da biste provjerili vašu implementaciju. Trebalo bi da greška bude reda `e-7` ili manje.

In [ ]:
N, D, H, T = 2, 5, 4, 3
x = np.linspace(-0.4, 0.6, num=N*T*D).reshape(N, T, D)
h0 = np.linspace(-0.4, 0.8, num=N*H).reshape(N, H)
Wx = np.linspace(-0.2, 0.9, num=4*D*H).reshape(D, 4 * H)
Wh = np.linspace(-0.3, 0.6, num=4*H*H).reshape(H, 4 * H)
b = np.linspace(0.2, 0.7, num=4*H)

h, cache = lstm_forward(x, h0, Wx, Wh, b)

expected_h = np.asarray([
 [[ 0.01764008,  0.01823233,  0.01882671,  0.0194232 ],
  [ 0.11287491,  0.12146228,  0.13018446,  0.13902939],
  [ 0.31358768,  0.33338627,  0.35304453,  0.37250975]],
 [[ 0.45767879,  0.4761092,   0.4936887,   0.51041945],
  [ 0.6704845,   0.69350089,  0.71486014,  0.7346449 ],
  [ 0.81733511,  0.83677871,  0.85403753,  0.86935314]]])

print('h greška: ', rel_error(expected_h, h))

# LSTM: prolaz unazad
Implementirajte prolaz unazad za LSTM kroz cijeli vremenski interval podataka u funkciji `lstm_backward` u fajlu `funkcije/rnn_layers.py`. 

Kada završite, pokrenite sledeću ćeliju da biste provjerili vašu implementaciju. Trebalo bi da greška bude reda `e-8` ili manje. (Za `dWh`, u redu je ako je greška reda `e-6` ili manje).

In [ ]:
from funkcije.rnn_layers import lstm_forward, lstm_backward
np.random.seed(231)

N, D, T, H = 2, 3, 10, 6

x = np.random.randn(N, T, D)
h0 = np.random.randn(N, H)
Wx = np.random.randn(D, 4 * H)
Wh = np.random.randn(H, 4 * H)
b = np.random.randn(4 * H)

out, cache = lstm_forward(x, h0, Wx, Wh, b)

dout = np.random.randn(*out.shape)

dx, dh0, dWx, dWh, db = lstm_backward(dout, cache)

fx = lambda x: lstm_forward(x, h0, Wx, Wh, b)[0]
fh0 = lambda h0: lstm_forward(x, h0, Wx, Wh, b)[0]
fWx = lambda Wx: lstm_forward(x, h0, Wx, Wh, b)[0]
fWh = lambda Wh: lstm_forward(x, h0, Wx, Wh, b)[0]
fb = lambda b: lstm_forward(x, h0, Wx, Wh, b)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
dh0_num = eval_numerical_gradient_array(fh0, h0, dout)
dWx_num = eval_numerical_gradient_array(fWx, Wx, dout)
dWh_num = eval_numerical_gradient_array(fWh, Wh, dout)
db_num = eval_numerical_gradient_array(fb, b, dout)

print('dx greška: ', rel_error(dx_num, dx))
print('dh0 greška: ', rel_error(dh0_num, dh0))
print('dWx greška: ', rel_error(dWx_num, dWx))
print('dWh greška: ', rel_error(dWh_num, dWh))
print('db greška: ', rel_error(db_num, db))

# LSTM model opisivanja

Sada kada ste implementirali LSTM, ažurirajte implementaciju `loss` metode u klasi `CaptioningRNN` u fajlu `funkcije/classifiers/rnn.py` za slučaj kada `self.cell_type` je  `lstm`. Ovo zahtijeva dodavanje manje od 10 linija koda.

Kada završite, pokrenite sledeću ćeliju da biste provjerili vašu implementaciju. Trebalo bi da greška bude reda `e-8` ili manje. 

In [ ]:
N, D, W, H = 10, 20, 30, 40
word_to_idx = {'<NULL>': 0, 'cat': 2, 'dog': 3}
V = len(word_to_idx)
T = 13

model = CaptioningRNN(word_to_idx,
          input_dim=D,
          wordvec_dim=W,
          hidden_dim=H,
          cell_type='lstm',
          dtype=np.float64)

# Set all model parameters to fixed values
for k, v in model.params.items():
    model.params[k] = np.linspace(-1.4, 1.3, num=v.size).reshape(*v.shape)

features = np.linspace(-0.5, 1.7, num=N*D).reshape(N, D)
captions = (np.arange(N * T) % V).reshape(N, T)

loss, grads = model.loss(features, captions)
expected_loss = 9.82445935443

print('funkcija cilja: ', loss)
print('očekivana vrijednost funkcije cilja: ', expected_loss)
print('razlika: ', abs(loss - expected_loss))

# Pretreniranje LSTM modela
Pokrenite sledeću ćeliju da biste overfit-ovali LSTM model na istom malom dataset-u koji ste koristili za RNN. Trebalo bi da konačna vrijednost funkcije cilje bude manja od 0.5.

In [ ]:
np.random.seed(231)

small_data = load_coco_data(max_train=50)

small_lstm_model = CaptioningRNN(
          cell_type='lstm',
          word_to_idx=data['word_to_idx'],
          input_dim=data['train_features'].shape[1],
          hidden_dim=512,
          wordvec_dim=256,
          dtype=np.float32,
        )

small_lstm_solver = CaptioningSolver(small_lstm_model, small_data,
           update_rule='adam',
           num_epochs=50,
           batch_size=25,
           optim_config={
             'learning_rate': 5e-3,
           },
           lr_decay=0.995,
           verbose=True, print_every=10,
         )

small_lstm_solver.train()

# Plot the training losses
plt.plot(small_lstm_solver.loss_history)
plt.xlabel('Iteracija')
plt.ylabel('Funkcija cilja')
plt.show()

# LSTM uzorci tokom vremena testiranja
Modifikujte `sample` metodu `CaptioningRNN` klase za slučaj kada je `self.cell_type` jednak `lstm`. 

Kada završite pokrenite sledeću ćeliju da biste uzorkovali iz pretreniranog LSTM modela na trening i validacionim primjerima. Kao i kod RNN-a, trening rezultati bi trebalo da budu veoma dobri, a validacioni rezultati vjerovatno neće imati smisla (zato što smo pretrenirali).

In [ ]:
for split in ['train', 'val']:
    minibatch = sample_coco_minibatch(small_data, split=split, batch_size=2)
    gt_captions, features, urls = minibatch
    gt_captions = decode_captions(gt_captions, data['idx_to_word'])

    sample_captions = small_lstm_model.sample(features)
    sample_captions = decode_captions(sample_captions, data['idx_to_word'])

    for gt_caption, sample_caption, url in zip(gt_captions, sample_captions, urls):
        plt.imshow(image_from_url(url))
        plt.title('%s\n%s\nGT:%s' % (split, sample_caption, gt_caption))
        plt.axis('off')
        plt.show()